# Launch Kokoro FastAPI on Google Colab

This notebook will fetch the Docker image, launch it, and provide an API URL for use in SillyTavern.

In [ ]:
# Install docker-py and pinggy
!pip install docker pinggy

In [ ]:
# Import necessary libraries
import docker
import time
from pinggy import Pinggy

In [ ]:
# Initialize Docker client
client = docker.from_env()

In [ ]:
# Pull the Docker image
image = client.images.pull('ghcr.io/remsky/kokoro-fastapi-cpu:v0.2.2')

In [ ]:
# Run the Docker container
container = client.containers.run(
    'ghcr.io/remsky/kokoro-fastapi-cpu:v0.2.2',
    ports={'8880/tcp': 8880},
    detach=True
)

In [ ]:
# Wait for the container to start
time.sleep(10)  # Adjust the sleep time if necessary

In [ ]:
# Obtain a public URL using Pinggy
pinggy = Pinggy()
public_url = pinggy.create_tunnel(8880)
api_url = public_url['url']
print(f'API URL: {api_url}')